In [38]:
import time
from datetime import datetime

import akshare as ak

In [39]:
def ma_calculate(df):
    ma_list = [5, 10, 20, 60, 120]

    for ma_len in ma_list:
        df['ma_' + str(ma_len)] = df['收盘'].rolling(ma_len).mean()

    return df.fillna(0)


def max_close_price_calculate(df):
    # 计算45个交易日的最高收盘价
    df['max_45'] = df['收盘'].rolling(45).max()
    return df


def detail_stock(symbol):
    """
    '日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率'
    :return:
    """
    time.sleep(0.1)  # 担心封ip
    print(symbol)
    today = datetime.now().strftime('%Y%m%d')
    # today = '20220827'
    stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date="20210510", end_date=today,
                                            adjust="qfq")

    df = ma_calculate(stock_zh_a_hist_df)
    df = max_close_price_calculate(df)

    # 偏离10日均线点数
    df['deviate_ma_10'] = (df['收盘'] - df['ma_10']) / df['收盘'] * 100
    # 偏离20日均线点数
    df['deviate_ma_20'] = (df['收盘'] - df['ma_20']) / df['收盘'] * 100
    
    # max price
    df['max_price'] = df[['开盘', '收盘']].max(axis=1)
    df['min_price'] = df[['开盘', '收盘']].min(axis=1)
    
    # 计算昨日收盘价
    df['昨收'] = df['收盘'] - df['涨跌额']

    # 上引线长度
    df['upper_lead'] = (df['最高'] - df['max_price']) / df['昨收'] * 100

    # 实体长度
    df['len_of_entity'] = abs(df['收盘'] - df['开盘']) / df['昨收'] * 100

    # 下引线长度
    df['lower_lead'] = (df['min_price'] - df['最低']) / df['昨收'] * 100

    return df

In [58]:
def filter_stock(ser):
    data = ser.to_dict()
    
    # 10MA > 20MA
    # 股价运行于季线之上
    cond1 = (data['ma_10'] >= data['ma_20']) and (data['收盘'] >= data['ma_60'])
    
    # 最高价高于45个交易日的收盘价
    cond2 = data['最高'] * 1.03 >= data['max_45']
    
    # 收盘价离10AM比较近（-4%内）,并且站上10日线的
    # 收盘价离20AM比较近（-8%内）,盈亏比考虑，生命线20日线不能太远
    cond3 = (-0.25 < data['deviate_ma_10'] <= 4) and (-0.25 < data['deviate_ma_20'] <= 8)
    
    
    # 换手率大于 2.8, 小于25 【真实的换手率其实需要排除掉一些实控人的资金，再计算的，这里不算严格】
    cond4 = (2.8 < data['换手率'] < 25)

    # 量比大于1.2 放量的, 这里没有量比
    # cond = data['量比'] > 1.2

    # 振幅大于 6.5
    # (-2 <= 涨跌幅 <= 5.5) [收在-2到5.5之间]
    # 小阳线
    cond5 = (data['振幅'] >= 6.5) and ((-2 < data['涨跌幅'] <= 5.5))

    # 上影线是实体的0.6倍以上,下引线小于2.3 【放宽条件，后续进行人工筛选】
    # 847 477
    # cond6 = (data['upper_lead'] >= data['len_of_entity'] * 0.8) and (data['lower_lead'] < 2.3)  
    # 591 396 0.67
    # cond6 = (data['upper_lead'] >= data['len_of_entity'] * 1.2) and (data['lower_lead'] < 2)
    # 386 313 0.815
    cond6 = (data['upper_lead'] >= data['len_of_entity'] * 1.5) and (data['lower_lead'] < 2)
    # cond6 = (data['upper_lead'] >= data['len_of_entity'] * 1.8) and (data['lower_lead'] < 1.2)

    
    return cond1 and cond2 and cond3 and cond4 and cond5 and cond6

# df = detail_stock('603366')
# df['filter'] = df.apply(test, axis=1)

# df.to_excel('test.xlsx', index=False)

In [59]:
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
df = stock_zh_a_spot_em_df[
    stock_zh_a_spot_em_df['代码'].str.startswith('00') | stock_zh_a_spot_em_df['代码'].str.startswith('60')]

code_list = df['代码'].tolist()
print(len(code_list))

3327


In [60]:
# load data from local
import os

files = os.listdir('daily_data/')
path_list = ['daily_data/'+f for f in files if f.endswith('.csv')]
f'lens: {len(files)}'

'lens: 3227'

In [61]:
# 第二天竞价入，第三天竞价出
import pandas as pd

start = time.time()
new_data = []
columns = ['symbol', 'date', 'profit']

count = 0

for path in path_list:

    symbol = path.split('/')[-1].split('.')[0]
    # print(path)
    df = pd.read_csv(path)
    #continue

    # filter符合条件的
    df['filter'] = df.apply(filter_stock, axis=1)
    
    # 支持十日线买入的

    index = df[df['filter']].index
    
    if index.empty:
        continue
    
    last_index = index.tolist()[-1]
    if last_index + 2 not in df.index:
        index = index[:-1]
    
    if last_index + 1 not in df.index:
        index = index[:-1]
    

    date_list = df['日期'].loc[index].tolist()

    next_price = df['开盘'].loc[index+1]
    
    # 第三天收盘卖
    # third_price = df['收盘'].loc[index+2]
    # 第三天开盘卖
    # third_price = df['开盘'].loc[index+2]
    
    # 第三天竞价，如果高开尾盘卖，低开不及预期开盘卖
    df['sale_price'] = df['收盘']
    df.loc[df['开盘'] < df['昨收'], 'sale_price'] = df['开盘'] 
    third_price = df['sale_price'].loc[index+2]

    res = (third_price.values - next_price.values)/df['昨收'].loc[index+1] * 100

    res = res.tolist()
    symbol_list = [symbol]*len(res)
    
    rows = list(zip(symbol_list, date_list, res))
    new_data += rows
    count += 1
    
    if count % 100 == 0:
        print(count)


ndf = pd.DataFrame(new_data, columns=columns)
ndf.to_excel('result_竞价情况.xlsx', index=False)

time.time() - start
# ndf
# 45 S

100
200
300
400
500
600
700
800
900
1000
1100
1200


178.35771322250366

In [56]:
# 第二天竞价入，第三天竞价出
import pandas as pd

new_data = []
columns = ['symbol', 'date', 'profit']

count = 0

for path in path_list:

    symbol = path.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(path)

    # filter符合条件的
    df['filter'] = df.apply(filter_stock, axis=1)

    index = df[df['filter']].index
    
    if index.empty:
        continue
    
    last_index = index.tolist()[-1]
    if last_index + 2 not in df.index:
        index = index[:-1]
    
    if last_index + 1 not in df.index:
        index = index[:-1]
    

    date_list = df['日期'].loc[index].tolist()

    next_price = df['开盘'].loc[index+1]
    
    # 第三天收盘卖
    third_price = df['收盘'].loc[index+2]
    # 第三天开盘卖
    # third_price = df['开盘'].loc[index+2]
    
    # 第三天竞价，如果高开尾盘卖，低开不及预期开盘卖
    #df['sale_price'] = df['收盘']
    #df.loc[df['开盘'] < df['昨收'], 'sale_price'] = df['开盘'] 
    #third_price = df['sale_price'].loc[index+2]

    res = (third_price.values - next_price.values)/df['昨收'].loc[index+1] * 100

    res = res.tolist()
    symbol_list = [symbol]*len(res)
    
    rows = list(zip(symbol_list, date_list, res))
    new_data += rows
    count += 1
    
    if count % 100 == 0:
        print(count)


ndf = pd.DataFrame(new_data, columns=columns)
ndf.to_excel('result_收盘.xlsx', index=False)
# ndf

100
200
300
400
500
600
700
800
900
1000


In [57]:
# 第二天竞价入，第三天竞价出
import pandas as pd

new_data = []
columns = ['symbol', 'date', 'profit']

count = 0

for path in path_list:

    symbol = path.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(path)

    # filter符合条件的
    df['filter'] = df.apply(filter_stock, axis=1)

    index = df[df['filter']].index
    
    if index.empty:
        continue
    
    last_index = index.tolist()[-1]
    if last_index + 2 not in df.index:
        index = index[:-1]
    
    if last_index + 1 not in df.index:
        index = index[:-1]
    

    date_list = df['日期'].loc[index].tolist()

    next_price = df['开盘'].loc[index+1]
    
    # 第三天收盘卖
    # third_price = df['收盘'].loc[index+2]
    # 第三天开盘卖
    third_price = df['开盘'].loc[index+2]
    
    # 第三天竞价，如果高开尾盘卖，低开不及预期开盘卖
    #df['sale_price'] = df['收盘']
    #df.loc[df['开盘'] < df['昨收'], 'sale_price'] = df['开盘'] 
    #third_price = df['sale_price'].loc[index+2]

    res = (third_price.values - next_price.values)/df['昨收'].loc[index+1] * 100

    res = res.tolist()
    symbol_list = [symbol]*len(res)
    
    rows = list(zip(symbol_list, date_list, res))
    new_data += rows
    count += 1
    
    if count % 100 == 0:
        print(count)


ndf = pd.DataFrame(new_data, columns=columns)
ndf.to_excel('result_开盘.xlsx', index=False)
# ndf

100
200
300
400
500
600
700
800
900
1000


In [74]:
# 十日线随缘买入法
import pandas as pd

start = time.time()
new_data = []
columns = ['symbol', 'date', 'profit']

count = 0

for path in path_list[:500]:

    symbol = path.split('/')[-1].split('.')[0]
    # print(path)
    df = pd.read_csv(path)
    #continue

    # filter符合条件的
    df['filter'] = df.apply(filter_stock, axis=1)
    
    # 支持十日线买入的
    df['on_10ma'] = False
    df.loc[df['最低'] <= df['ma_10'], 'on_10ma'] = True
    df['on_10ma'] = df['on_10ma'].shift(-1)
    

    index = df[df['filter'] & df['on_10ma']].index
    
    if index.empty:
        continue
    
    last_index = index.tolist()[-1]
    if last_index + 2 not in df.index:
        index = index[:-1]
    
    if last_index + 1 not in df.index:
        index = index[:-1]
    

    date_list = df['日期'].loc[index].tolist()

    next_price = df['开盘'].loc[index+1]
    
    # 第三天收盘卖
    # third_price = df['收盘'].loc[index+2]
    # 第三天开盘卖
    # third_price = df['开盘'].loc[index+2]
    
    # 第三天竞价，如果高开尾盘卖，低开不及预期开盘卖
    df['sale_price'] = df['收盘']
    df.loc[df['开盘'] < df['昨收'], 'sale_price'] = df['开盘'] 
    third_price = df['sale_price'].loc[index+2]

    res = (third_price.values - next_price.values)/df['昨收'].loc[index+1] * 100

    res = res.tolist()
    symbol_list = [symbol]*len(res)
    
    rows = list(zip(symbol_list, date_list, res))
    new_data += rows
    count += 1
    
    if count % 100 == 0:
        print(count)


ndf = pd.DataFrame(new_data, columns=columns)
# ndf.to_excel('result_B10_竞价情况.xlsx', index=False)

time.time() - start
# ndf
# 45 S
ndf.mean()

100


/var/folders/nd/8l9b_3p15xx7dn65b45mywc40000gn/T/ipykernel_95212/1476058596.py:72: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ndf.mean()


symbol         inf
profit    0.105305
dtype: float64

In [75]:
# 昨日涨停的去掉
# 实体阴线太长的去掉
# 非常靠近均线尝试
ndf

,symbol,date,profit
0,603596,2020-06-30,-4.520744
1,002362,2022-06-21,-1.819505
2,002404,2020-06-19,4.107425
3,000561,2020-06-12,5.807365
4,003040,2021-12-17,0.485437
...,...,...,...
188,600846,2019-04-02,7.978723
189,603383,2020-06-22,-0.198265
190,002775,2022-04-11,4.072398
191,601999,2022-01-11,-1.881331
